Install required libraries and tools

In [ ]:
!pip install rouge.score nltk py7zr
!pip install accelerate -U
!pip install transformers -U
!pip install datasets
!pip install torch torchvision torchaudio
!pip install sentencepiece

# note: you may need to restart your runtime after installation

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 7.1 MB/s eta 0:00:00
  Created wheel for rouge.score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=ebf7c96fe0592ff861314b38f18af19dec09c444a40be8702d68cb1d17acf70d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge.score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import transformers
import datasets
from datasets import load_dataset, load_metric, load_from_disk, load_metric, Dataset
import numpy as np
import nltk
from transformers import BartTokenizer, BartForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#### Upload required dataset, we will be using the Amazon Fine Food Reviews [Amazon Fine Food Reviews](https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews)dataset from Kaggle!

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving reviews_clean5.csv to reviews_clean5.csv


For this notebook, we will be using 70000 reviews only due to computational constraints. We will proceed to do some final cleaning on our already cleaned dataset.

We set the maximum text length and summary length to be 150 and 10 respectively, and filter out rows that does not fulfil this criteria.

In [ ]:
df = pd.read_csv('reviews_clean5.csv',nrows=70000)
pd.set_option('display.max_colwidth',None)
df.drop_duplicates(subset=['Text'], inplace=True)
df.dropna(axis=0,inplace=True)
df.replace('',np.nan,inplace=True)

max_text_len=150
max_summary_len=10

cleaned_text =np.array(df['Text'])
cleaned_summary=np.array(df['Summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])

df=pd.DataFrame({'text':short_text,'summary':short_summary})

def remove_short(text):
  tokens = text.split()
  longs = []
  for i in tokens:
     if len(i)>1:
        longs.append(i)
  return (" ".join(longs))

clean_summary = []
for t in df['summary']:
   clean_summary.append(remove_short(t))

df['summary'] = clean_summary

df = df[df['summary'].str.len()>=2]

In [ ]:
df['id'] = df.index
df

,text,summary,id
0,i have bought several of the vitality canned dog food products and have found them all to be of good quality the product looks more like a stew than a processed meat and it smells better my labrador is finicky and she appreciates this product better than most,good quality dog food,0
1,product arrived labeled as jumbo salted peanutsthe peanuts were actually small sized unsalted not sure if this was an error or if the vendor intended to represent the product as jumbo,not as advertised,1
2,this is a confection that has been around a few centuries it is a light pillowy citrus gelatin with nuts in this case filberts and it is cut into tiny squares and then liberally coated with powdered sugar and it is a tiny mouthful of heaven not too chewy and very flavorful i highly recommend this yummy treat if you are familiar with the story of cs lewis the lion the witch and the wardrobe this is the treat that seduces edmund into selling out his brother and sisters to the witch,delight says it all,2
3,if you are looking for the secret ingredient in robitussin i believe i have found it i got this in addition to the root beer extract i ordered and made some cherry soda the flavor is very medicinal,cough medicine,3
4,great taffy at a great price there was a wide assortment of yummy taffy delivery was very quick if your a taffy lover this is a deal,great taffy,4
...,...,...,...
58809,this brand not only lives up to its name but it also brews very well there is no bitter after taste and it leaves one feeling very satisfied only after one cup,rich but very satisfying,58809
58810,for years i have been a huge starbucks coffee lover devoted infact but i was out of coffee and the small grocery store i went to did not carry starbucks so i debated what to get and grab this brand i waited a few days to try it i was sceptic but now i am in love with gevalia coffee and will probably buy it from now onit is smooth tasteful and beyond heaven,the best,58810
58811,this gevalia one of my favorites coffee good flavor and aroma and good price hope that it had more aroma but still great,favorite coffee,58811
58812,i really like gevalia colombian roast it has so rich aroma and great taste one of my favorite drink it several times a day,great coffee,58812


In [ ]:
metric = load_metric('rouge')
train, validate, test = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])


<ipython-input-6-373b65c73954>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')


In [ ]:
train = Dataset.from_dict(train)
val = Dataset.from_dict(validate)
test = Dataset.from_dict(test)
df = datasets.DatasetDict({'train':train,'validate':val,'test':test})
df

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'id'],
        num_rows: 47043
    })
    validate: Dataset({
        features: ['text', 'summary', 'id'],
        num_rows: 5880
    })
    test: Dataset({
        features: ['text', 'summary', 'id'],
        num_rows: 5881
    })
})

In [ ]:
def preprocess_data(data_to_process):
  inputs = [reviews for reviews in data_to_process['text']]
  #tokenize the reviews
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize the summaries
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary'], max_length=max_target, padding='max_length', truncation=True)

  #set labels
  model_inputs['labels'] = targets['input_ids']
  #return the tokenized data
  #input_ids, attention_mask and labels
  return model_inputs

In [ ]:
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoint ='t5-small'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

pad_on_right = tokenizer.padding_side == "right"

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model,label_pad_token_id=tokenizer.pad_token_id)

In [ ]:
max_input = 150
max_target = 10
batch_size = 8

In [ ]:
tokenize_data = df.map(preprocess_data, batched = True,remove_columns=['id','text','summary'])
tokenize_data

Map:   0%|          | 0/47043 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/5880 [00:00<?, ? examples/s]

Map:   0%|          | 0/5881 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 47043
    })
    validate: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5880
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5881
    })
})

In [ ]:
#####################
# metrics
# compute rouge for evaluation
#####################

def compute_rouge(pred):
  #print(pred)
  predictions, labels = pred

  #decode the predictions
  predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)

  #decode labels
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [ ]:
import torch
import gc

gc.collect()

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("[INFO] training using {}".format(torch.cuda.get_device_name(0)))
#print('There are %d GPU(s) available.' % torch.cuda.device_count())

[INFO] training using Tesla T4


In [ ]:
torch.cuda.empty_cache()
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    #f"{model_name}-finetuned-newsarticles",
    output_dir='./t5modelnew',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True
)
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenize_data['train'],  # Your preprocessed training dataset
    eval_dataset=tokenize_data['validate'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.887800,1.779306,16.516500,5.113600,16.302900,16.319400,4.219200
2,1.865500,1.746369,17.210800,5.413200,17.018200,17.037900,4.205100
3,1.823000,1.738303,17.125100,5.297100,16.929800,16.934600,4.228900


TrainOutput(global_step=17643, training_loss=1.945363497548935, metrics={'train_runtime': 2175.7654, 'train_samples_per_second': 64.864, 'train_steps_per_second': 8.109, 'total_flos': 5595894462873600.0, 'train_loss': 1.945363497548935, 'epoch': 3.0})

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.797800,1.708794,16.904300,5.243400,16.685100,16.709800,4.261400
2,1.773600,1.697028,17.484400,5.582300,17.271000,17.300000,4.247400
3,1.757900,1.692441,17.683800,5.707500,17.431200,17.453700,4.266300


TrainOutput(global_step=17643, training_loss=1.7790646988984875, metrics={'train_runtime': 2138.2438, 'train_samples_per_second': 66.002, 'train_steps_per_second': 8.251, 'total_flos': 5595894462873600.0, 'train_loss': 1.7790646988984875, 'epoch': 3.0})

In [ ]:
predict_results = trainer.predict(
            tokenize_data['test'],max_length=150, num_beams=3)

In [ ]:
metrics = predict_results.metrics

In [ ]:
metrics

{'test_loss': 1.682761788368225,
 'test_rouge1': 16.7562,
 'test_rouge2': 6.3087,
 'test_rougeL': 16.5039,
 'test_rougeLsum': 16.4737,
 'test_gen_len': 7.4525,
 'test_runtime': 295.5655,
 'test_samples_per_second': 19.897,
 'test_steps_per_second': 2.49}

In [ ]:
predictions = np.where(predict_results.predictions != -100, predict_results.predictions, tokenizer.pad_token_id)
predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)
predictions = [pred.strip() for pred in predictions]

In [ ]:
for i in range(2000,2100):
    print("Review:",df['test']['text'][i])
    print("Original summary:",df['test']['summary'][i])
    print("Predicted summary:",predictions[i])
    print("\n")

Review: this  pb  tastes  good  and  is  goood  for  you it  makes  good  sandwiches i  only  wish  it  were  in  stock  more  often
Original summary: tasty
Predicted summary: good pb


Review: a  tennis  player  hubby  of  mine  got  this  pack  on  the  rack  of  his  opel  corsa  and  casually i started  cherishing  the  peppermint  flavour it  is  got  to  lasting  refreshing  flavour  but  the  best  part  is its  real  mouth  freshner  and  above  all keeps  my  teeth  so  whiter  and  feels  good  chewing  for  a long  time normally indians  prefer mukhwas masalas after  lunch  or  dinner  but  i prefer  the  chewing  gum  as  it  refreshes  the  taste  buds good  pick
Original summary: refreshing mouth freshner
Predicted summary: mukhwas masalas


Review: this  is  a great  tasting  snack full  of  things  that  are  healthy a  little  pricey obviously but  no  more  expensive  than  a pint  of  ice  cream iherb  is  one  of  the  best  online  merchants  around as  well order 

In [ ]:
predictions[2020:2030]

['moonstruck chocolates',
 'my dogs prefer the wingaling',
 'greatsleep',
 'lipton tea',
 'great vegan lunch',
 'great popcorn',
 'great toy',
 'ghirardelli hot cocoa',
 'smells like rotting fish',
 'my favorite chewing gum']

In [ ]:
df['test']['summary'][2020:2030]

['moonstruck chocolates',
 'dogs vote it thumbs up',
 'pretty good',
 'what deal',
 'healthy and quick',
 'outstanding popcorn',
 'love the fact that it comes with ball',
 'ghirardelli chocolate mocha hot cocoa 16 oz can',
 'my daughter calls them pukesbring the nose plugs',
 'my favorite gum']

In [ ]:
df['test']['text'][2020:2030]

['the  first  time  i tried  moonstruck  chocolates  i was  hooked these  are  the  very  best  chocolate  truffles  i  ordered  the  truffle  box  and  it  arrived  quickly  everyone  should  try  a moonstruck  chocolate it  is  really  a treat ',
 'my  two  dogs  prefer  the  wingaling  over  the  other  varieties  of  this  dog  food there  are  actual  wings  packed  into  the  can  along  with  other  meat  chunks veggies  and  liquid there  is  a bit  more  liquid  than  i would  like  but  my  dogs  eat  it it  does  not  go  to  waste  and  you  can  see  that real meat  is  used',
 'at  first i  did  not  think  this  stuff  was  incredible but  someone  told  me  to  mix  it  with  68  oz  of  hot  milk  or  12  water and  12  milkit  was  sooooooo  much  better  like  that i  have  been  drinking  the  line  of  og  products  now  for  5 months and  i feel  greatsleep  better on  12  the  dose  of  prozac  i have  been  on  for  8 years and  just  have  more  energy the  mil

In [ ]:
import shutil
model.save_pretrained("/content/t5model")
tokenizer.save_pretrained("/content/tokenizert5")
shutil.make_archive('/content/t5model_new','zip','/content/t5model')

shutil.make_archive('/content/t5tokenizer_new','zip','/content/tokenizert5')

from google.colab import files
files.download('/content/t5model_new.zip')